In [23]:
import vdblite
import numpy as np
from time import time
from uuid import uuid4
import sys
from pprint import pprint as pp
import dataset
import pickle
from subgraph import query_to_kg_embeddings

In [5]:
file = open("simple_wikidata5m.pkl", "rb")
model = pickle.load(file)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings
alias2entity = dataset.wikidata5m.load_alias("./entity.txt.gz")

In [6]:
entity_db = vdblite.Vdb()
id2entity = model.graph.id2entity

In [7]:
entity2alias = {v: k for k, v in alias2entity.items()}

In [8]:
print(type(entity2alias))
for i in entity2alias:
    print(i, entity2alias[i])
    break

<class 'dict'>
Q5196650 cut your hair


In [9]:
for i in range(len(entity_embeddings)):
    entity_db.add({'entity_embedding': entity_embeddings[i], 'entity_id': i})

In [10]:
entity_db.details()

DB elements #: 4818298
DB size in memory: 39064728 bytes


In [27]:
# Pass the output of query_to_kg_embeddings
# 
def get_neighbourhood(query_embeddings):
    neighbours = {}
    for qe in query_embeddings:
        nearest = entity_db.search(query_embeddings[qe], field='entity_embedding', count=5)
        print("Nearest: ", len(nearest))
        for n in nearest:
            try:
                alias = entity2alias[id2entity[n["entity_id"]]]
                neighbours[alias] = id2entity[n["entity_id"]]
            except Exception as e:
                pass
    return neighbours

In [15]:
temp = entity_embeddings[entity2id[alias2entity["steve jobs"]]]
query_embedding = {"steve jobs": temp}

In [16]:
neighbours = get_neighbourhood(query_embeddings=query_embedding)
for n in neighbours:
    print(n, neighbours[n])

Nearest:  5
steve jobs dies Q19837
laurene jobs Q3133593
chrisann brennan Q18810921
dr. ed catmull Q93161
eric emerson schmidt Q92747


In [17]:
import os
import tqdm
halueval_path = os.path.join(os.getcwd(), "submodules", "HaluEval", "data", "qa_data.json")
print(halueval_path)

/Users/apoorvamk/Desktop/CSE8803/KG-LLM-Hallucination/submodules/HaluEval/data/qa_data.json


In [18]:
import json
with(open(halueval_path, "r")) as f:
    data_lst = f.readlines()

data_lst = [json.loads(data) for data in data_lst]

In [19]:
RELEVANT_KEYS = ['knowledge', 'question']

In [21]:
def process_data(data):
    result = {}
    for k in RELEVANT_KEYS:
        query_embedding = query_to_kg_embeddings(data[k])
        neighbours = get_neighbourhood(query_embedding)
        result.update(neighbours)
    return result

In [28]:
processed_data = [process_data(i) for i in data_lst[:2]]

KeyError:  'the 19th century'
KeyError:  'first'
KeyError:  'usa'
KeyError:  'arthur magazine 1844 1846 literary periodical published 19th century first women woman magazine published usa'
KeyError:  'arthur magazine 1844 1846 literary periodical published 19th century first women woman magazine published'
KeyError:  'magazine 1844 1846 literary periodical published 19th century first women woman magazine published usa'
KeyError:  'arthur magazine 1844 1846 literary periodical published 19th century first women woman magazine'
KeyError:  'magazine 1844 1846 literary periodical published 19th century first women woman magazine published'
KeyError:  '1844 1846 literary periodical published 19th century first women woman magazine published usa'
KeyError:  'arthur magazine 1844 1846 literary periodical published 19th century first women woman'
KeyError:  'magazine 1844 1846 literary periodical published 19th century first women woman magazine'
KeyError:  '1844 1846 literary periodical publ

In [31]:
print(len(processed_data))
print(data_lst[0], processed_data[0])

2
{'knowledge': "Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.First for Women is a woman's magazine published by Bauer Media Group in the USA.", 'question': "Which magazine was started first Arthur's Magazine or First for Women?", 'right_answer': "Arthur's Magazine", 'hallucinated_answer': 'First for Women was started first.'} {'men and women (newspaper supplement)': 'Q16895083', 'abhidhanantar': 'Q16057964', 'tung padevat': 'Q7852993', 'portals of prayer': 'Q7231514', 'image magazine (ireland)': 'Q6002116', '4-4-0st': 'Q2037045', '4-6-0t': 'Q483711', '0-6-0st': 'Q2922269', 'pacific locomotive': 'Q1043481', 'mogul (locomotive)': 'Q2441836', 'philadelphia commuter rail': 'Q1345', 'st. lewis, mo': 'Q38022', 'new your city': 'Q60', 'cin city': 'Q43196', 'san fransico, california': 'Q62', 'heinrich bauer verlag kg': 'Q554198', 'bbc magazines': 'Q16994752', 'motor trend group': 'Q7565129', 'daughters of bilitis': 'Q1145872',

In [ ]:
from tqdm.contrib.concurrent import process_map
import multiprocessing as mp
import time
mp.cpu_count()
start = time.time()
result = process_map(process_data, data_lst, max_workers=5)
for data in tqdm.tqdm(data_lst):
    result.append(process_data(data))
end = time.time()
print(end-start)